# Notebook

## Data Collection
The data collection part of the program comes from highway cameras. While the logic and codebase for managing the cameras will not be shown. What can be seen is one specific camera and the capturing of data before the connection starts and after. The reason for keeping track before is because the stream is constantly running and we want to keep track of whenever the camera is actually recording.

It is also important to note that these cameras exist pretty much in a while true loop but it does have logic for a backoff algorithm if the camera stops recording. It is also important to note that the total camera amount currently is ~3000.

We have two classes of log events. One which logs when a camera starts and one which logs when a camera stops. When a camera stops we have an additional three categories; camera stopped (no more segments to read), manual stop (stop_event), and error (camera stopped recording due to an error). The manual stop happens when the camera managing application wants to manually stop the camera but the two other errors are due to some camera error. The error is a bit more serious and points to a problem with debugging the segments and or HTTP request issues like 404 or 500.

Before starting the recording the camera:

In [ ]:
put_log_event({
    "type": "start",
    "time": current_time,
    "proxyUrl": url,
})

After recording the camera:

In [ ]:
if stop_event.is_set():
    put_log_event({
        "type": "stopped due to priority change",
        "time": current_time,
    })
elif process.returncode != 0:
    put_log_event({
        "type": "error",
        "time": current_time,
        "proxyUrl": url,
        "error": stderr.decode("utf-8"),
    })
else:
    put_log_event({
        "type": "stop",
        "time": current_time,
    })

The actual recording part is simple and requires two lines of code:

In [ ]:
cmd = [
    "ffmpeg",
    "-i", "https://44-fte.divas.cloud/CHAN-5758/CHAN-5758_1.stream/playlist.m3u8?{ip}:{token}",
    "-c", "copy",
    "-map", "0",
    "-f", "segment",
    "-segment_time", "60",
    "-segment_format", "mpegts",
    "-strftime", "1",
    f"{os.path.join(BASE_DIR, data['cameraId'])}\{current_time}_output_%Y-%m-%d_%H-%M-%S.ts",
    "-nostats",
    "-loglevel", "warning",
]

env = os.environ.copy()
env['HTTP_PROXY'] = proxy_url
env['HTTPS_PROXY'] = proxy_url

process = subprocess.Popen(cmd, env=env, stderr=subprocess.PIPE)
_, stderr = process.communicate()

The function for logging the data can then be simply defined in the following code segment. We can see that we have a global PartitionKey for all of the cameras because we want to process the cameras together. In theory, this grouping could be changed to example regions like Florida, New York, etc. because the analysis performed of the data will probably be different for different regions.

In [9]:
def put_log_event(data):
    kinesis_client.put_record(
        StreamName="pktraffic-camera-stream",
        Data=json.dumps(data),
        PartitionKey="cameras"
    )

## Kinesis Stream

Setting up the kinesis stream is extremely easy and consists of a few steps. The first step is simply to navigate to the kinesis stream service on aws console and register a new stream. During the creation process, you get to specify a few configurations such as the number of shards and the retention period. For our case, we use on-demand mode and a retention period of 24 hours.

The goal of the kinesis stream is just to batch the data together and then write it to different databases. This means that we do not need to have a longer retention period than the time it takes to write the data to the databases.

![kinesis](images/kineses.png)

In [ ]:
kinesis_client = boto3.client('kinesis')

To test that it actually works we write a test event to the stream:

In [ ]:
put_log_event({
    "type": "start",
    "time": "test",
    "proxyUrl": "test",
})

![kinesis](images/example.png)

## Lambda Function

The goal of the lambda function is to manage the data from the kinesis stream and write it to the databases. The lambda function is written in Python and is triggered by the kinesis stream. The lambda will be triggered if there is a total of 100 records or if the data is older than 5 minutes. This is to ensure that the data is written to the databases in a timely manner. Understand that these limits are for the specific case where ~3,000 cameras are running. These configurations will change where in the future this application should be able to handle upwards of 50,000 - 100,000 cameras.

To save money, rather than saving entire error code messages like an HTTP request we have a mapping of errors. This does increase the complexity but in this specific case, it is worth it. The logic for mapping errors is not shown but mostly consists of trying to map the error to a specific error code.

In [ ]:
1: "HTTP error 404 Not Found"
2: "Camera Stopped Recording"
2: "Immediate exit requested"
3: "Camera Started Recording"
4: "Unknown error"
5: "End of file"
6: "HTTP error 403 Forbidden"
7: "HTTP error 503 Service Unavailable"
8: "I/O error"
9: "SEI type 5"
10: "expired from playlists"
12: "co located POCs unavailable"
13: "Non-monotonous DTS in output stream"
14: "missing picture in access unit with size"
15: "Manual Stop"
16: "Exception"
17: "Stopped due to priority change"
18: "Unrecognized error"
19: "Empty error"

The analysis required consists of two different parts, dynamoDB storing of the log and the current status of the camera in MySQL. In the future, there will be more analysis performed such as why cameras stopped recording (the entire region stopped).

Keep in mind that the lambda code shown is just a simplified version without the error mapping, database connection, error handling, etc. For example, there is also another logic for handling the case where we didn't not receive an expected error (HTTP error) but rather a different error such as error codes 18, and 16.

In [11]:
def store_records(records):
    write_requests = []

    for record in records:
        payload = json.loads(record['kinesis']['data'])

        camera_id = payload['cameraId']
        current_time = payload['time']
        status = payload['status']

        write_requests.append({
            'PutRequest': {
                'Item': {
                    "c": {"S": camera_id},
                    "t": {"S": current_time},
                    "s": {"N": str(status)},
                }
            }
        })
    
    requests = {"pktraffic-camera-logging": write_requests}
    
    while requests:  # BatchWriteItem can only do 25 requests at a time
        response = dynamo_client.batch_write_item(RequestItems=requests)
        requests = response.get('UnprocessedItems', {})

In [ ]:
def store_status(records):
    for record in records:
        payload = json.loads(record['kinesis']['data'])

        camera_id = payload['cameraId']
        current_time = payload['time']
        status = payload['status']

        sql = """
            UPDATE camera-status
            SET lastUpdated = %s, status = %s
            WHERE cameraId = %s
        """

        cursor.execute(sql, (current_time, status, camera_id))
        connection.commit()

In [ ]:
def lambda_handler(event, context):
    store_status(event['Records'])
    store_records(event['Records'])

    return {
        'statusCode': 200,
        'body': json.dumps('Batch write completed!')
    }


## Storing Data

The data will mainly be stored in two different ways. A NoSQL database to store the history of cameras and a relational database to store the current status of the cameras. The reason for this is that the relational database is better suited for the current status of the cameras and the noSQL database is better suited for the history of the cameras.

### DynamoDB

The purpose of using dynamoDB is to store the history of the cameras. Meaning that it should be possible to backtrack to any given time if the camera was recording and if not the reason it wasn't recording. This means that we will in most cases want to query the data based on a specific camera ID and a specific time and or time range. Utilizing dynamoDB with the help of PK (camera) and sk (timestamp) we can easily and efficiently query the data.

![dynamoDB](images/dynamo2.png)

After running the data collection system for about a week we end up with some interesting statistics. In the specific case we can see extremely big storage saves by utilizing an error mapping where each item is around 33.7 bytes.

![dynamoDB](images/stat.png)

### MySQL

The second database is a MySQL database which is stored on AWS. The purpose is to store the latest status of the cameras. This means that we mainly only need to store three different columns; cameraId, status, and timestamp. We then can mainly query the entire table to get the latest status of all the cameras.

## Dashboard
The dashboard will consist of two different parts. The first part is a geographical map of the cameras and the second part is individual cameras. The geographical map will be used to visualize the cameras that are currently running and the second part is a dashboard displaying a specific camera and its history. Due to the time limitations of this project, only the second part will be completed. The dashboard for a specific camera will consist of the inputs as a specific date and a camera id. We then will get the status of the camera +- 3 days of the specific date. The dashboard is made so that it becomes extremely easy to explore a specific camera during different iterations to see why or why not it was recording.

The dashboard implementation will be using an aws lambda endpoint and react.

The lambda endpoint calculations for getting the logs consist of two main queries. The first query will get all longs within a specific timeframe and the second query will get the first record before a date using a couple constrains (specify the order of data returned, limit amount of data returned etc). The lambda than also contains some logic for managing and formatting the return data in the correct format.

In [ ]:
def fetch_data(camera_id, start_date_str, end_date_str):
    response = dynamodb.query(
        TableName='pktraffic-camera-logging',
        KeyConditionExpression='c = :camera_id AND t BETWEEN :start_date AND :end_date',
        ExpressionAttributeValues={
            ':camera_id': {'S': camera_id},
            ':start_date': {'S': start_date_str},
            ':end_date': {'S': end_date_str}
        }
    )

def fetch_first_before_date(camera_id, start_date_str):
    response = dynamodb.query(
        TableName='pktraffic-camera-logging',
        KeyConditionExpression='c = :camera_id AND t < :start_date',
        ExpressionAttributeValues={
            ':camera_id': {'S': camera_id},
            ':start_date': {'S': start_date_str}
        },
        ScanIndexForward=False,
        Limit=1
    )


![dashboard](images/dashboard.jpg)

## Dump code

Here are a few more code snippets of the project like the frontend and backend code for the dashboard.

### Backend Lambda

In [ ]:
# camera_logs/camera_logs.py
import boto3
import datetime
from boto3.dynamodb.types import TypeDeserializer
from concurrent.futures import ThreadPoolExecutor
import pytz


def fetch_data(camera_id, start_date_str, end_date_str):
    dynamodb = boto3.client('dynamodb')

    response = dynamodb.query(
        TableName='pktraffic-camera-logging',
        KeyConditionExpression='c = :camera_id AND t BETWEEN :start_date AND :end_date',
        ExpressionAttributeValues={
            ':camera_id': {'S': camera_id},
            ':start_date': {'S': start_date_str},
            ':end_date': {'S': end_date_str}
        }
    )
    deserializer = TypeDeserializer()
    items = response.get('Items', [])
    return [{k: deserializer.deserialize(v) for k, v in item.items()} for item in items]


def fetch_first_before_date(camera_id, start_date_str):
    dynamodb = boto3.client('dynamodb')

    response = dynamodb.query(
        TableName='pktraffic-camera-logging',
        KeyConditionExpression='c = :camera_id AND t < :start_date',
        ExpressionAttributeValues={
            ':camera_id': {'S': camera_id},
            ':start_date': {'S': start_date_str}
        },
        ScanIndexForward=False,
        Limit=1
    )

    deserializer = TypeDeserializer()
    item = response.get('Items', [{}])
    if item:
        item = item[0]
    return {k: deserializer.deserialize(v) for k, v in item.items()} if item else {'c': camera_id, "t": "2023-01-01T00:00:00Z", "s": 0}


def florida_to_utc_str(fl_date_str):
    """Convert Florida time string to UTC string."""
    fl_date = datetime.datetime.strptime(fl_date_str, '%Y-%m-%d')
    local_dt = pytz.timezone('US/Eastern').localize(fl_date)
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt.strftime('%Y-%m-%d')


def get_traffic_data(camera_id, date_str):
    target_date_utc_str = florida_to_utc_str(date_str)
    target_date_utc = datetime.datetime.strptime(target_date_utc_str, '%Y-%m-%d').replace(tzinfo=pytz.utc)

    delta = datetime.datetime.now(pytz.timezone('US/Eastern')) - target_date_utc

    if delta.days <= 3:
        end_date = datetime.datetime.now(pytz.timezone('US/Eastern'))
        start_date = end_date - datetime.timedelta(days=7)
    else:
        start_date = target_date_utc - datetime.timedelta(days=3)
        end_date = target_date_utc + datetime.timedelta(days=4)

    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    with ThreadPoolExecutor(max_workers=2) as executor:
        data_future = executor.submit(fetch_data, camera_id, start_date_str, end_date_str)
        first_before_future = executor.submit(fetch_first_before_date, camera_id, start_date_str)

        data = data_future.result()
        first_before = first_before_future.result()

    return data, first_before


def process_data(data, first_before):
    grouped_data = {}

    florida_tz = pytz.timezone('US/Eastern')

    for item in data:
        timestamp_utc = datetime.datetime.strptime(item['t'], '%Y-%m-%dT%H:%M:%SZ')
        timestamp_utc = pytz.utc.localize(timestamp_utc)
        timestamp_florida = timestamp_utc.astimezone(florida_tz)

        date_str = timestamp_florida.strftime('%Y-%m-%d')
        if date_str not in grouped_data:
            grouped_data[date_str] = []
        grouped_data[date_str].append(item)

    result = {}
    prev_log = first_before

    sorted_days = sorted(grouped_data.keys())

    for day in sorted_days:
        sorted_logs = sorted(grouped_data[day], key=lambda x: x['t'])

        florida_midnight = datetime.datetime.strptime(day, '%Y-%m-%d').replace(hour=0, minute=0, second=0)
        florida_midnight = pytz.timezone('US/Eastern').localize(florida_midnight)

        utc_equivalent = florida_midnight.astimezone(pytz.utc)

        if sorted_logs[0]['t'] != utc_equivalent.strftime('%Y-%m-%dT%H:%M:%SZ') and prev_log:
            result[day] = [{
                'c': prev_log['c'],
                's': prev_log['s'],
                't': utc_equivalent.strftime('%Y-%m-%dT%H:%M:%SZ')
            }] + sorted_logs
        else:
            result[day] = sorted_logs

        prev_log = sorted_logs[-1]

    if len(sorted_days) == 0:
        compare_date = first_before['t'].split('T')[0]
    else:
        compare_date = sorted_days[0]

    start_date = datetime.datetime.strptime(compare_date, '%Y-%m-%d')
    for i in range(7):
        date_str = (start_date + datetime.timedelta(days=i)).strftime('%Y-%m-%d')
        if date_str not in result:
            result[date_str] = []

    return result


def adjust_log(log):
    log['s'] = int(log['s']) if log['s'] % 1 == 0 else float(log['s'])
    utc_dt = datetime.datetime.strptime(log['t'], '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=datetime.timezone.utc)
    florida_dt = utc_dt.astimezone(pytz.timezone('US/Eastern'))
    log['t'] = florida_dt.strftime('%Y-%m-%d %H:%M:%S')

    return log


def adjust_logs(logs_by_date):
    for date, logs in logs_by_date.items():
        for i, log in enumerate(logs):
            logs_by_date[date][i] = adjust_log(log)
    return logs_by_date


def filter_data(data):
    def str_to_datetime(s):
        return datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S')

    def remove_close_events(events):
        filtered_events = []
        prev_event_time = None
        for event in reversed(events):
            current_event_time = str_to_datetime(event['t'])
            if (
                not prev_event_time
                or (prev_event_time - current_event_time).total_seconds() > 300
            ):
                filtered_events.append(event)
            prev_event_time = current_event_time
        return list(reversed(filtered_events))

    def remove_consecutive_same_s(events):
        filtered_events = []
        prev_s = None
        for event in events:
            if event['s'] != prev_s:
                filtered_events.append(event)
            prev_s = event['s']
        return filtered_events

    for date, events in data.items():
        events = remove_close_events(events)
        events = remove_consecutive_same_s(events)
        data[date] = events

    return data


def get_data(event):
    data, first_before = get_traffic_data(event["cameraId"], event["date"])
    processed_data = process_data(data, first_before)
    adjusted_logs = adjust_logs(processed_data)

    return filter_data(adjusted_logs), adjusted_logs


In [ ]:
# camera_logs/handler.py
from json import loads, dumps
from camera_logs.camera_logs import get_data


def lambda_handler(event, context):
    data = loads(event['body']) if 'body' in event else event

    print("event", data)

    information, unfiltered_information = get_data(data)

    print("information", information)
    print("unfiltered_information", unfiltered_information)

    return {
        "statusCode": 200,
        "headers": {
            "Access-Control-Allow-Origin": "*",
        },
        "body": dumps({
            "data": {
                "information": information,
                "unfilteredInformation": unfiltered_information,
            }
        })
    }


In [ ]:
# serverless.yml
service: district-service

frameworkVersion: '3'

provider:
  name: aws
  runtime: python3.9
  region: us-east-1
  stage: main
  environment:
    ...
  iamManagedPolicies:
    - "arn:aws:iam::aws:policy/AmazonS3FullAccess"
    - "arn:aws:iam::aws:policy/AmazonCognitoPowerUser"
    - "arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess"

package:
  individually: true
  exclude:
    - ./**

functions:
  ...
  cameraLogs:
    timeout: 30
    handler: camera_logs/handler.lambda_handler
    description: Camera API endpoint for district service
    memorySize: 128
    events:
      - http:
          path: camera-logs
          method: post
          cors: true
          authorizer:
            name: cognito-authorizer
            arn: arn:aws:cognito-idp:${self:provider.region}:${self:provider.environment.ACCOUNT_ID}:userpool/${self:provider.environment.COGNITO_POOL_ID}
    package:
      include:
        - camera_logs/**
    layers:
      ...

### Frontend Dashboard

Excludes all of the Redux code.

In [ ]:
import React, { useEffect } from 'react';
import { createUseStyles } from 'react-jss';
import { BaseProps } from '../../Models/Props';
import { Box, Paper, TextField } from '@mui/material';
import { LocalizationProvider, DatePicker } from '@mui/x-date-pickers';
import { AdapterDayjs } from '@mui/x-date-pickers/AdapterDayjs';
import { GET_CAMERA_LOGS, SET_CAMERA, SET_CAMERA_CAMERAID, SET_CAMERA_DATE } from '../../Redux/Actions';
import { Theme } from '../../Styling/Theme';
import { getCamera } from '../../Redux/Selectors';
import { useSelector } from 'react-redux';
import dayjs from 'dayjs';

const useStyles = createUseStyles((theme: Theme) => {
  return {
    timelineContainer: {
      position: 'relative',
      width: 'calc(100% - 40px)',
      height: 'calc(100vh - 50px)',
      display: 'flex',
      flexDirection: 'column',
      margin: '0 20px',
      overflowX: "hidden",
    },
    segmentWrapper: {
      flex: 1,
      position: 'relative',
      transform: 'scale(0.98)',
    },
    baseLine: {
      width: '100%',
      height: '1px',
      backgroundColor: 'gray',
    },
    greenLine: {
      position: 'absolute',
      height: '5px',
      backgroundColor: 'green',
      top: '10px',
    },
    redLine: {
      position: 'absolute',
      height: '5px',
      backgroundColor: 'red',
      top: '0',
    },
    timeLabel: {
      position: 'absolute',
      top: '25px',
      fontSize: '10px',
      whiteSpace: 'nowrap',
      transform: 'rotate(-30deg)',
      transformOrigin: 'top right',
      '&:before': {
        content: '""',
        position: 'absolute',
        top: '-5px',
        left: '50%',
        width: '1px',
        height: '5px',
        backgroundColor: 'gray',
        transform: 'rotate(30deg)',
      },
    },
    timeLine: {
      position: 'absolute',
      top: '20px',
      width: '100%',
      height: '1px',
      backgroundColor: 'gray',
      marginTop: '5px',
    },
    segment: {
      position: 'absolute',
      height: '20px',
      top: '0',
      '&:hover $segmentLabel': {
        visibility: 'visible',
      },
    },
    segmentLabel: {
      visibility: 'hidden',
      position: 'absolute',
      bottom: '100%',
      whiteSpace: 'nowrap',
      backgroundColor: 'white',
      padding: '5px 10px',
      border: '1px solid black',
      borderRadius: '5px',
      boxShadow: '0px 0px 5px rgba(0,0,0,0.1)',
      color: 'black',
      fontSize: '12px',
    },
    verticalLine: {
      position: 'absolute',
      width: '2px',
      height: '30px',
      backgroundColor: 'gray',
      top: '-5px',
      zIndex: 1,
    },
    errorBox: {
      position: 'absolute',
      backgroundColor: 'white',
      padding: '2px',
      border: '1px solid black',
      whiteSpace: 'nowrap',
      fontSize: '10px',
      textAlign: 'center',
      transform: 'translateX(-50%)',
      '&:hover $errorLabel': {
        visibility: 'visible',
      },
    },
    arrow: {
      position: 'absolute',
      width: '0',
      height: '0',
      borderTop: '5px solid white',
      borderLeft: '5px solid transparent',
      borderRight: '5px solid transparent',
    },
    errorLabel: {
      visibility: 'hidden',
      position: 'absolute',
      bottom: '100%',
      whiteSpace: 'nowrap',
      backgroundColor: 'white',
      padding: '2px 5px',
      border: '1px solid black',
    },
    timeLineLabel: {
      marginTop: -22,
    },
    boxLayout: {
      display: "flex",
      justifyContent: "center",
      gap: 20,
      padding: "20px 0 40px 0",
      "& > div": {
        width: 350,
        height: 52,
      },
      "& .react-daterange-picker__calendar": {
        zIndex: 10,
      },
      "& .react-daterange-picker__inputGroup__input": {
        color: theme.textPrimary,
      },
      "& .react-daterange-picker__wrapper": {
        borderRadius: 4,
        borderColor: theme.borderMUI,
      },
      "& .react-daterange-picker__inputGroup:first-child": {
        textAlign: "right",
      },
      "@media (max-width: 600px)": {
        flexDirection: "column",
        "& > div": {
          width: "100%",
        },
      },
    }
  }
});

const errorMapping: { [key: number]: string } = {
    1: "HTTP error 404 Not Found",
    2: "Stopped Recording (No data received)",
    3: "Recording",
    4: "Unknown error",
    5: "End of file",
    6: "HTTP error 403 Forbidden",
    7: "HTTP error 503 Service Unavailable",
    8: "I/O error",
    9: "SEI type 5",
    10: "expired from playlists",
    12: "co-located POCs unavailable",
    13: "Non-monotonous DTS in output stream",
    14: "missing picture in access unit with size",
    15: "Manual Stop",
    16: "Exception",
    17: "Stopped due to priority change",
    18: "Unrecognized error",
    19: "Empty error",
  };

export const Camera: React.FC<BaseProps> = ({ dispatch }): JSX.Element => {
  const camera = useSelector(getCamera);

  const classes = useStyles();

  useEffect(() => {
		window.scrollTo(0, 0);
		const urlParams = new URLSearchParams(window.location.search);
    dispatch({ type: SET_CAMERA, payload: {
      cameraId: urlParams.get("cameraId"),
      date: urlParams.get("date"),
      information: {},
      unfilteredInformation: {},
    } });
	}, [dispatch]);

  useEffect(() => {
    if (!camera.cameraId || !camera.date) return;
    dispatch({ type: GET_CAMERA_LOGS, payload: null });
  }, [dispatch, camera.cameraId, camera.date]);

  const calculateWidthPercentage = (start: string, end: string) => {
    const startMoment = new Date(start).getTime();
    const endMoment = new Date(end).getTime();
    const oneDayInMilliseconds = 24 * 60 * 60 * 1000;

    return ((endMoment - startMoment) / oneDayInMilliseconds) * 100;
  };

  const handleHover = (e: React.MouseEvent, log: any, start: string, end: string, i: number, index: number) => {
    const segmentLabel = document.getElementById(`segmentLabel-${index}-${i}`);
    if (!segmentLabel) return;
  
    segmentLabel.style.visibility = 'visible';
  };

  const renderSegments = (logs: any, index: number) => {
    const segments = [];
    for (let i = 0; i < logs.length; i++) {
      const start = logs[i].t;
      let end: any;
      if (logs.length === 1) {
        end = new Date(new Date(start).getTime() + 1000 * 60 * 60 * 48).toISOString().replace(/T.*/, 'T00:00:00');
      } else {
        end = i < logs.length - 1 ? logs[i + 1].t : new Date(new Date(logs[logs.length - 1].t).getTime() + 1000 * 60 * 60 * 24).toISOString().replace(/T.*/, 'T00:00:00');
      }
      const width = calculateWidthPercentage(start, end);
      const color = logs[i].s === 3 ? 'green' : 'red';
  
      // Calculate the position of the left side of the segment
      const startDate = new Date(start);
      const secondsSinceMidnight = 
        startDate.getSeconds() + 
        (startDate.getMinutes() * 60) + 
        (startDate.getHours() * 3600);
      const percentOfDay = (secondsSinceMidnight / (24 * 3600)) * 100;
      
      if (i !== 0) {
        segments.push(
          <div
            key={`line-${i}`}
            className={classes.verticalLine}
            style={{ left: `${percentOfDay}%` }}
          />
        );
      }

      segments.push(
        <div
          key={`segment-${index}-${i}`}
          className={classes.segment}
          style={{ left: `${percentOfDay}%`, width: `${width}%`, backgroundColor: color }}
          onMouseOver={(e) => handleHover(e, logs[i], start, end, i, index)}
          onMouseOut={() => {
            const segmentLabel = document.getElementById(`segmentLabel-${index}-${i}`);
            if (!segmentLabel) return;
            segmentLabel.style.visibility = 'hidden';
          }}
        >
          <div id={`segmentLabel-${index}-${i}`} className={classes.segmentLabel}>
            {errorMapping[logs[i].s] || 'Unmapped error'}: {new Date(start).toLocaleTimeString()} to {new Date(end).toLocaleTimeString()}
          </div>
        </div>
      );
    }
    return segments;
  };  

  const renderTimeLabels = () => {
    return new Array(24).fill(null).map((_, hour) => {
      let displayHour = hour % 12 || 12;
      const amPm = hour >= 12 ? 'PM' : 'AM';
      return (
        <span 
          key={hour} 
          className={classes.timeLabel} 
          style={{ left: `${(hour / 24) * 100}%` }}
        >
          {`${displayHour} ${amPm}`}
        </span>
      );
    });
  }

  return (
    <Paper className={classes.timelineContainer}>
      <Box className={classes.boxLayout}>
        <TextField
          label="Camera ID"
          variant="outlined"
          value={camera.cameraId}
          onChange={(value) => {
            dispatch({ type: SET_CAMERA_CAMERAID, payload: { cameraId: value.target.value } });
          }}
        />
        <LocalizationProvider dateAdapter={AdapterDayjs}>
          <DatePicker
            label="Date"
            value={dayjs(camera.date)}
            onChange={(date) => {
              if (!date) return;
              const dateMod = `${date.year()}-${String(date.month() + 1).padStart(2, '0')}-${String(date.date()).padStart(2, '0')}`;
              dispatch({ type: SET_CAMERA_DATE, payload: dateMod  });
            }}
          />
        </LocalizationProvider>
      </Box>
      {Object.keys(camera.information).length && Object.keys(camera.information).sort((a, b) => Date.parse(a) - Date.parse(b)).map((date, index) => (
        <div 
          key={date} 
          className={classes.segmentWrapper}
        >
          <h3 className={classes.timeLineLabel}>
            {new Date(date).toLocaleDateString()}
          </h3>
          <div className={classes.timeLine} />
          {renderSegments(camera.information[date], index)}
          {renderTimeLabels()}
        </div>
      ))}
    </Paper>
  );
};